# what

This is going to be me doing a section extraction of the reports.

For some of the rpeorts there will be regex section extra section of the paragraph numbers.

However for many of the reports this is not possible so I will perform a more traditional document chunking technique. However I do need to have a unique ID for each section

In [ ]:
import pandas as pd
import engine.extract.ReportExtracting as ReportExtracting
import shutil
import os
import re
import numpy as np
import importlib
from tqdm import tqdm
importlib.reload(ReportExtracting)

tqdm.pandas()

# Current state

In [ ]:
report_text = pd.read_pickle(
    '../../output/parsed_reports.pkl'
)
report_text

In [ ]:
extractor = ReportExtracting.ReportExtractingProcessor('../../output/parsed_reports.pkl')
extractor.extract_sections_from_text(15, 'report_section.pkl')

In [ ]:
report_section = pd.read_pickle('report_section.pkl').merge(report_text, on='report_id')[['report_id', 'text', 'sections']].set_index('report_id')  
report_section['num_sections'] = report_section['sections'].map(lambda x: len(x))
report_section['has_sections'] = report_section['num_sections'] > 0
report_section['year'] = report_section.index.map(lambda x: int(x.split('_')[2]))
report_section['agency'] = report_section.index.map(lambda x: x.split('_')[0])
report_section['avg_section_length'] = report_section['sections'].map(lambda x: np.mean(x['section_text'].map(len)) if )
report_section

# Reports without matches

In [ ]:
missing_sections = report_section[report_section['has_sections'] == False]
display(missing_sections['year'].describe())
display(missing_sections['agency'].value_counts())
missing_sections

Most of the reports are not being matched by the current report section extractor.

I will need to confirm that there are not report that are being missed by the report extractor.

In [ ]:
missing_sections.loc['ATSB_a_2000_072']['paragraphs']

In [ ]:
# Manually looking at all of the report pdfs and see if theyt exist
if os.path.exists('reports_to_look_at'):
    shutil.rmtree('reports_to_look_at')
os.mkdir('reports_to_look_at')
for report in missing_sections.sample(frac=0.01, random_state=42).index:
    shutil.copyfile(f'../../output/report_pdfs/{report}.pdf', f'reports_to_look_at/{report}.pdf') 
    with open(f'reports_to_look_at/{report}.txt', 'w') as f:
        f.write(missing_sections.loc[report]['text'])
    with open(f'reports_to_look_at/{report}_sections.txt', 'w') as f:
        f.write('\n\n'.join([f'{paragraph_name}\n{paragraph}' for _, paragraph_name, paragraph in missing_sections.loc[report]['paragraphs'].itertuples()]))

## Looking at current taic sections

I want to know how  big the sections are becuase maybe I could just split up into pages. Or try to do paragraphs. However I know that paragraphsare going to be harder because random spaces are always added.

In [ ]:
taic_sections = report_section[(report_section['has_sections'] == True) & (report_section['agency'] == 'TAIC')]
display(taic_sections['year'].hist())
display(taic_sections['agency'].value_counts())

In [ ]:
pd.concat(taic_sections['sections'].tolist())['section_text'].map(len).describe()

In [ ]:
other_sections = report_section[(report_section['num_sections'] > 4)  & (report_section['agency'] != 'TAIC')]
display(other_sections['year'].hist())
display(other_sections['agency'].value_counts())
pd.concat(other_sections['sections'].tolist())['section_text'].map(len).describe()

In [ ]:
print("TAIC sections average section length")
display(taic_sections['sections'].map(lambda x: x['section_text'].map(len).mean()).describe())
print("Taic sections average number of sections")
taic_sections['sections'].map(len).describe()

In [ ]:
print("Other sections average section length")
display(other_sections['sections'].map(lambda x: x['section_text'].map(len).mean()).describe())
print("Other sections aaverage number of sections")
other_sections['sections'].map(len).describe()

In [ ]:
def inspect(dir, df):
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.mkdir(dir)
    for report in df.index:
        shutil.copyfile(f'../../output/report_pdfs/{report}.pdf', f'{dir}/{report}.pdf') 
        with open(f'{dir}/{report}.txt', 'w') as f:
            f.write(df.loc[report]['text'])
        with open(f'{dir}/{report}_sections.txt', 'w') as f:
            f.write('\n\n'.join([f'{section}\n{section_text}' for _, section, section_text, _ in df.loc[report]['sections'].itertuples()]))


inspect('other_section', other_sections.sample(10, random_state=42))

THe autowork of the section extractors has not worked on the other agencies reports. This is problematic.

## Looking at using pages

In [ ]:
def extract_pages(text):
    page_matches = list(re.finditer(r'<< Page (\d+|[xvi]+) >>', text))

    pages = []
    for page_match in range(len(page_matches)-1):

        page = page_matches[page_match].group(1)

        pages.append({
            "page": page,
            "text": text[page_matches[page_match].start():page_matches[page_match+1].start()],
        })

    return pd.DataFrame(pages)

In [ ]:
missing_sections['pages'] = missing_sections['text'].map(extract_pages)
missing_sections

In [ ]:
pd.concat(missing_sections['pages'].tolist())['text'].map(len).describe()

The problem here is that they are about 2 and half times longer so they are not splitting it up very well.

## Looking at paragraph splitting

I am not sure if this will work but will have alook at it

In [ ]:
def split_into_paragraphs(text):
    raw_splits = [paragraph.strip() for paragraph in re.split(r'\n *\n', text) if len(paragraph.strip()) > 0]

    splits_df = pd.DataFrame(raw_splits, columns=['paragraph'])
    
    splits_df['page'] = splits_df['paragraph'].map(lambda x: re.match(r'<< Page (\d+|[xvi]+) >>', x).group(1) if re.match(r'<< Page (\d+|[xvi]+) >>', x ) else None)
    splits_df.ffill(inplace=True)
    splits_df.replace({np.nan: '0', None: '0'}, inplace=True)
    splits_df['paragraph_num'] = splits_df.groupby(['page']).cumcount()
    splits_df['paragraph_name'] = 'p' + splits_df['page'] + "." + splits_df['paragraph_num'].astype(str)

    splits_df = splits_df[splits_df['paragraph'].map(lambda x: len(re.sub(r'<< Page (\d+|[xvi]+) >>', '', x).strip()))>8]

    return splits_df[['paragraph_name', 'paragraph']]

In [ ]:
missing_sections['paragraphs'] = missing_sections['text'].map(split_into_paragraphs)

In [ ]:
pd.concat(missing_sections['paragraphs'].tolist())['paragraph'].map(len).describe()

This seems to be working alright and is of reasonable length.